In [2]:
import pandas as pd
import numpy as np
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine

In [4]:
#ingestamos datasets

Amazon = pd.read_csv('Datasets/amazon_prime_titles.csv') #9668 registros
Disney = pd.read_csv('Datasets/disney_plus_titles.csv') #1450 registros
Hulu = pd.read_csv('Datasets/hulu_titles.csv') #3073 registros
Netflix = pd.read_json('Datasets/netflix_titles.json') #8807 registros

In [5]:
#insertamos columna con la plataformas correspondientes

Amazon['Platform'] = 'Amazon'
Disney['Platform'] = 'Disney'
Hulu['Platform'] = 'Hulu'
Netflix['Platform'] = 'Netflix'

In [6]:
#Construimos dataset con todas las plataformas
Total_Data = pd.concat([Amazon, Disney, Hulu, Netflix])
Total_Data.reset_index(drop=True, inplace=True)

In [7]:
Total_Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22998 entries, 0 to 22997
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       22998 non-null  object
 1   type          22998 non-null  object
 2   title         22998 non-null  object
 3   director      14739 non-null  object
 4   cast          17677 non-null  object
 5   country       11499 non-null  object
 6   date_added    13444 non-null  object
 7   release_year  22998 non-null  int64 
 8   rating        22134 non-null  object
 9   duration      22516 non-null  object
 10  listed_in     22998 non-null  object
 11  description   22994 non-null  object
 12  Platform      22998 non-null  object
dtypes: int64(1), object(12)
memory usage: 2.3+ MB


In [9]:
#Eliminamos columnas que no necesitaremos para la consigna requerida
DF = Total_Data.drop(['show_id', 'country', 'director', 'date_added', 'rating', 'description'], axis=1)

In [10]:
#convertimos el tipo de dato de la columna 'duration' a datos numericos
DF['duration'] = DF['duration'].str.replace('min' , '')
DF['duration'] = DF['duration'].str.replace('Seasons' , '')
DF['duration'] = DF['duration'].str.replace('Season' , '')
DF['duration'] = pd.to_numeric(DF['duration'])


In [114]:
# consulta de prueba n°1
Filtro = (DF['Platform'] == 'Hulu') & (DF['release_year'] == 2018) & (DF['type'] == 'Movie')
elemento = DF[Filtro]['duration'].idxmax()
DF.loc[elemento,'title']

'The House That Jack Built'

In [106]:
# consulta de prueba n°2
condicion = (DF['Platform']=='Netflix')
Nro_Movies = DF[condicion]['type'].str.contains(pat = 'Movie').sum()
Nro_TVShows = DF[condicion]['type'].str.contains(pat ='TV Show').sum()
print(Nro_Movies,Nro_TVShows)

6131 2676


In [68]:
#Consulta de prueba n°3
Filtro = DF['listed_in'].str.contains('Comedy')
Nro_Comedias = DF[Filtro].groupby('Platform')['listed_in'].count()
print(Nro_Comedias)

Platform
Amazon     2099
Disney      526
Hulu        667
Netflix     399
Name: listed_in, dtype: int64


In [125]:
#Consulta de prueba n°4

condicion = (DF['Platform']=='Netflix') & (DF['release_year']==2018) 
actores = DF[condicion]['cast'].str.split(pat =',', expand=True)
union = pd.concat(objs = [actores[i] for i in range(len(actores.columns))], ignore_index = True)
union.value_counts()

 Andrea Libman        8
 Rupa Bhimani         6
 Vincent Tong         6
 Jigna Bharadhwaj     6
 Takahiro Sakurai     6
                     ..
 T.J. Miller          1
 Beau Knapp           1
 Géraldine Pailhas    1
 Richard Alan Reid    1
 Tom Holland          1
Length: 6825, dtype: int64

In [101]:
#Generamos Base de Datos

engine =sqlalchemy.create_engine('sqlite:///Datasets/filmes.db')
TD_1.to_sql(name='filmes', con = engine) 


22998

In [102]:
#Verificamos la lectura del DF a partir de la base de datos
pd.read_sql('filmes', engine)

,index,type,title,cast,release_year,duration,listed_in,Platform
0,0,Movie,The Grand Seduction,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",2014,113.0,"Comedy, Drama",Amazon
1,1,Movie,Take Care Good Night,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",2018,110.0,"Drama, International",Amazon
2,2,Movie,Secrets of Deception,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",2017,74.0,"Action, Drama, Suspense",Amazon
3,3,Movie,Pink: Staying True,"Interviews with: Pink, Adele, Beyoncé, Britney...",2014,69.0,Documentary,Amazon
4,4,Movie,Monster Maker,"Harry Dean Stanton, Kieran O'Brien, George Cos...",1989,45.0,"Drama, Fantasy",Amazon
...,...,...,...,...,...,...,...,...
22993,22993,Movie,Zodiac,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",2007,158.0,"Cult Movies, Dramas, Thrillers",Netflix
22994,22994,TV Show,Zombie Dumb,None,2018,2.0,"Kids' TV, Korean TV Shows, TV Comedies",Netflix
22995,22995,Movie,Zombieland,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",2009,88.0,"Comedies, Horror Movies",Netflix
22996,22996,Movie,Zoom,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",2006,88.0,"Children & Family Movies, Comedies",Netflix


La consigna no contempla la implementación de un modelo de ML, sino solamente la realización de consultas, por lo que se decide no eliminar registros con datos faltantes.